# Reranking

<img src="./media/reranking.png" width=600>

[Image: Rerankers and Two-Stage Retrieval](https://www.pinecone.io/learn/series/rag/rerankers/)

As semantic similarity becomes a core technique for delivering context to LLM-based applications, the challenge of **finding truly relevant information** grows more important. Most modern systems use **embedding models** to convert unstructured text into vector representations, storing these in a vector database for fast similarity-based retrieval.

While this first-step retrieval process is efficient and scalable, **the top results may not always be the best-aligned passages for a given query.** They might be “near matches,” but not the most contextually relevant.
This is where **reranking** comes in: a second-stage process that reorders the initially retrieved set to better match the true information need.

Reranking uses a dedicated model—typically a **cross-encoder** or a **late interaction model** to directly compare each candidate passage with the query, assigning a fine-grained relevance score. By re-evaluating these candidate passages, rerankers help surface  the most useful, specific, and accurate results to the top.

In this notebook, we’ll explore the most popular reranking approaches in modern RAG pipelines, with an intuitive look at how these models work and how they improve retrieval quality.

---
## Reranking Models

<img src="./media/embedding.png" width=600>

[Choosing the Right Embedding Model for RAG in Generative AI](https://medium.com/bright-ai/choosing-the-right-embedding-for-rag-in-generative-ai-applications-8cf5b36472e1)

In a two stage RAG pipeline, we rely on a few different pre-trained encoder models to convert our unstructured content (generally text) into dense vector representations that capture the learned semantics of language through scaled machine learning. The first of which is the commonly known "Embedding Model" set up as a bi-encoder, one for the query one for the document(s). As this notebook is meant to focus on reranking, I will just provide a brief overview of base embedding models in this context, but if you'd like to learn the specifics you can check out [my guide on bidirectional encoder representations from transformers](https://www.youtube.com/watch?v=n_UQ0e0fBIA)!

### Context: Bi-Encoder (Embedding Model)

<img src="./media/biencoder.png" width=300>

[Cross-encoders vs Bi-encoders : A deep-dive into text encoding methods](https://medium.com/@rbhatia46/cross-encoders-vs-bi-encoders-a-deep-dive-into-text-encoding-methods-d9aa890d6ca4)

The **bi-encoder architecture** is widely used for vector similarity search in retrieval systems, where the base embedding model independently encodes queries and documents into numerical vectors (“embeddings”) that capture their meaning. At search time, these embeddings are quickly compared (usually via cosine similarity) to find the most relevant matches. This dual encoder architecture is where the "bi" from bi-encoder comes from, and typically uses models like [BERT](https://arxiv.org/pdf/1810.04805), which leverage the transformer architecture and attention mechanism to learn powerful language representations via objectives like masked language modeling (MLM). In MLM, the model is trained to predict missing words in sentences, allowing it to develop a deep understanding of context and semantics after training on many millions of examples, allowing the model to perform this dense vector encoding.

As a quick clarification, the “bi” in bi-encoder refers to using **two separate encoding passes** (one for queries, one for documents), often with shared model weights which is separate from BERT’s “bidirectional” attention, which enables each token to attend to both left and right context during training.

Let's take a quick look at what this looks like with a popular embedding model [all-MiniLM-L6-v2](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2)

#### Load the Model
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
#### Define the Query & Documents
query = "Who is the best technical AI youtuber?"

documents = [
    "The dogwood is the state flower of North Carolina",
    "Adam Lucek makes videos about artificial intelligence",
    "The canada goose has a lifespan of 10-24 years"]
#### Example Encoding

Here we can see what happens when the text is encoded into a numerical form. In the case of the embedding model we're using, our sentence will be mapped to a 384 dimensional dense vector space.
query_embedding = embedding_model.encode(query)

print("First 10 dimensions: ", query_embedding[:10])
print("\n Total Size: ",len(query_embedding), "dimensions.")
#### Embedding Documents Independently
doc_embeddings = embedding_model.encode(documents)
#### Computing Similarity
similarity = embedding_model.similarity(query_embedding, doc_embeddings)

print(similarity)
We can see that our query: *Who is the best technical AI youtuber?* is most similar to the embedding in position 2 (index 1): *Adam Lucek makes videos about artificial intelligence* 😎

This is the backbone of bi-encoder based vector similarity based retrieval, converting and storing document embeddings then comparing them at run time to embedded queries. This done a lot more efficiently than our quick example using [vector databases](https://github.com/ALucek/embeddings-guide/blob/main/WTF_VDB.ipynb), but supports our first stage of the two stage RAG process- initial retrieval. Once our retrieved set is available, we can then employ our reranking models to provide a more refined ranking of relevant context for query responses.
### Cross Encoder

<img src="./media/cross-encoder.png" width=800>

[The Illustrated Guide to Cross-Encoders: From Deep to Shallow](https://medium.com/@kakumar1611/the-illustrated-guide-to-cross-encoders-from-deep-to-shallow-2a23a8630016)

The **cross-encoder architecture** extends base embedding models by training them as classifiers for direct semantic similarity or relevance. Rather than computing document and query embeddings independently for later comparison, cross-encoders take the query and each document and **concatenate them as a single input** (e.g., `[CLS] query [SEP] document [SEP]`) to the model. The output is a **direct relevance score** for the query-document pair, typically produced by the model’s classification \[CLS] token (or an output head attached to it).

The key advantage of the cross-encoder is that **every token in the input (query and document) can attend to every other token**, allowing for richer, fine-grained interactions between query and document. This usually yields higher accuracy and more nuanced matching than bi-encoders. However, this comes at a computational cost as each query-document pair must be processed together, which is slower than similarity search with precomputed embeddings, hence why it's often applied as the second step in retrieval after a smaller candidate set has been retrieved.
#### Load the Model

For this example we'll be using the popular cross-encoder [ms-marco-MiniLM-L6-v2](https://huggingface.co/cross-encoder/ms-marco-MiniLM-L6-v2)
from sentence_transformers import CrossEncoder

cross_encoder = CrossEncoder("cross-encoder/ms-marco-MiniLM-L6-v2")
#### Define the Query & Documents
query = "What are the health benefits of meditation?"

documents = [
    "Several clinical studies have shown that regular meditation can help reduce stress and anxiety levels.",
    "Meditation involves focusing the mind and eliminating distractions, often through breathing techniques or guided imagery.",
    "A daily meditation practice has been associated with lower blood pressure and improved sleep quality in adults.",
    "The city of Kyoto is famous for its Zen temples, where meditation has been practiced for centuries.",
    "People who meditate frequently often report feeling calmer and more focused throughout the day.",
    "Research suggests meditation may lower the risk of heart disease by reducing inflammation and improving heart rate variability.",
    "Meditation apps have become increasingly popular, offering guided sessions on mindfulness and relaxation.",
    "A 2021 meta-analysis found that meditation can reduce symptoms of depression when used alongside other treatments.",
    "Some forms of meditation emphasize compassion and kindness, aiming to improve emotional well-being.",
    "Athletes sometimes use meditation techniques to enhance concentration and mental resilience during competition.",
]
#### Rank Documents
ranks = cross_encoder.rank(query, documents)

print("="*25, "Cross Encoder Rankings", "="*25, "\n")
for rank in ranks:
    print(f"{rank['score']:.2f}\t{documents[rank['corpus_id']]}")
As you can see we're able to order the passages a lot more closer to what's relevant to our query! Whereas a regular bi-encoder retrieval may focus too much on keywords and themes we're able to clearly extract the specifics.
### Late Interaction Model

<img src="./media/multi.png" width=800>

[ColBERT: Efficient and Effective Passage Search via Contextualized Late Interaction over BERT](https://arxiv.org/pdf/2004.12832)

The late interaction architecture came after the popularization of using a cross-encoder with the release of ColBERT research and introduces a unique approach inspired by both the bi-encoder and cross-encoder setup. Late interaction based reranking takes the set of retrieved documents and query and splits each into their respective tokens. The query and each candidate document are then encoded as a matrix of token level vectors. This is opposed to the pooled single vector representation that is output by a base embedding model that combines all the token level embeddings together. Then for each query token the similarity is compared to all document token vectors and the maximum value is kept. This maximum similarity is aggregated across all query tokens to produce the final relevancy score for each document.
<img src="./media/late-interaction.excalidraw.png" width=800>

Mathematically, given:

- Query $Q$ with tokens $q_1, q_2, \ldots, q_m$
- Candidate document $D$ with tokens $d_1, d_2, \ldots, d_n$

Each token is encoded into a vector: $Q = [\mathbf{q}_1, \mathbf{q}_2, \ldots, \mathbf{q}_m]$, $D = [\mathbf{d}_1, \mathbf{d}_2, \ldots, \mathbf{d}_n]$.

For each query token $\mathbf{q}_j$: $s_j = \max_k \left( \cos (\mathbf{q}_j, \mathbf{d}_k) \right)$

The final document relevance score $S$ is aggregated by sum or mean:
$S = \sum_{j=1}^m s_j$, or $S = \frac{1}{m} \sum_{j=1}^m s_j$
#### Ranking with ColBERT

We'll be using [ColBERT V2](https://huggingface.co/colbert-ir/colbertv2.0) for our demonstration. Along with the model is an [official repo](https://github.com/stanford-futuredata/ColBERT/tree/main) that includes specific abstractions and functions meant to process documents and ranking with ColBERT and the proposed late interaction approach. But for demonstrations sake we'll implement late interaction directly with `transformers` and `torch`.
#### Load the Model

We'll use the [transformers](https://huggingface.co/docs/transformers/en/index) package directly to load the tokenizer and model from the 🤗 Hub.
from transformers import AutoTokenizer, AutoModel

colbert_tokenizer = AutoTokenizer.from_pretrained("colbert-ir/colbertv2.0")
colbert = AutoModel.from_pretrained("colbert-ir/colbertv2.0")
#### Define the Query & Documents

Same from before!
query = "What are the health benefits of meditation?"

documents = [
    "Several clinical studies have shown that regular meditation can help reduce stress and anxiety levels.",
    "Meditation involves focusing the mind and eliminating distractions, often through breathing techniques or guided imagery.",
    "A daily meditation practice has been associated with lower blood pressure and improved sleep quality in adults.",
    "The city of Kyoto is famous for its Zen temples, where meditation has been practiced for centuries.",
    "People who meditate frequently often report feeling calmer and more focused throughout the day.",
    "Research suggests meditation may lower the risk of heart disease by reducing inflammation and improving heart rate variability.",
    "Meditation apps have become increasingly popular, offering guided sessions on mindfulness and relaxation.",
    "A 2021 meta-analysis found that meditation can reduce symptoms of depression when used alongside other treatments.",
    "Some forms of meditation emphasize compassion and kindness, aiming to improve emotional well-being.",
    "Athletes sometimes use meditation techniques to enhance concentration and mental resilience during competition.",
]
#### Helper Functions

We'll be defining two helper functions, `get_token_embeddings` and `colbert_score`.

`get_token_embeddings` will compute the token-level embeddings through the model, taking in text and first running it through the tokenizer to split into individual tokens. Those tokens are then sent through the model to create a vector representation for each. We remove any special classifier or separator tokens that may have been output and keep just the individual embeddings.

`colbert_score` then takes in the query embeddings and document embeddings, which have the shape of matrices:

* The **query embeddings** have shape \$(m, d)\$, where \$m\$ is the number of query tokens and \$d\$ is the embedding dimension.
* The **document embeddings** have shape \$(n, d)\$, where \$n\$ is the number of document tokens.

The scoring function computes the **cosine similarity** between each query token embedding and every document token embedding, resulting in an \$(m, n)\$ similarity matrix. For each query token, we take the **maximum similarity** value across all document tokens. These maximum similarities are then **summed** across all query tokens to produce the final relevance score for the document.
import torch
import torch.nn.functional as F

def get_token_embeddings(text, tokenizer, model):
    # Get token-level embeddings, ignore [CLS] and [SEP]
    inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=128)
    
    with torch.no_grad():
        outputs = model(**inputs)
    
    # outputs.last_hidden_state: (1, seq_len, hidden_dim)
    input_ids = inputs['input_ids'][0]
    keep_indices = (input_ids != tokenizer.cls_token_id) & (input_ids != tokenizer.sep_token_id)
    
    return outputs.last_hidden_state[0][keep_indices]  # (filtered_seq_len, hidden_dim)

def colbert_score(query_emb, doc_emb):
    
    # query_emb: (m, d), doc_emb: (n, d)
    sim = F.cosine_similarity(query_emb.unsqueeze(1), doc_emb.unsqueeze(0), dim=2)  # (m, n)
    
    max_sim, _ = sim.max(dim=1)  # (m,)
    
    return max_sim.sum().item()
#### Rank Documents

Now we put it all together by embedding the query and each document, computing the scores, then sorting the documents by relevance score
# Compute token level embeddings
query_emb = get_token_embeddings(query, colbert_tokenizer, colbert)
doc_embs = [get_token_embeddings(doc, colbert_tokenizer, colbert) for doc in documents]

# Compute scores
scores = [colbert_score(query_emb, doc_emb) for doc_emb in doc_embs]

ranking = sorted(zip(scores, documents), reverse=True)

print("="*25, "Late Interaction Rankings", "="*25, "\n")
for score, doc in ranking:
    print(f"{score:.2f}\t{doc}")
<img src="./media/colbert_evals.png" width=800>

In the ColBERT research, they found that this late interaction approach provided comparable performance to existing cross encoders and other reranking methods but was quicker and required much less computation!
## Putting it Together

Now that we have an understanding of the models and approaches for reranking, let's create a simple RAG pipeline that can query and rerank results for a RAG response
### Vector Database Setup
#### Text Chunking

For our database we'll do a simplified chunking setup on [The Adventures of Sherlock Holmes](https://www.gutenberg.org/ebooks/1661) as available through Project Gutenberg. We'll grab the text from the website and setup a very simple recursive token chunker that first splits the text into sentences, then combines into chunks of roughly 400 tokens long. These will be our candidate chunks for retrieval embedded into our vector database!
import requests
import nltk
import tiktoken

# Download Sherlock Holmes text
url = "https://www.gutenberg.org/files/1661/1661-0.txt"
response = requests.get(url)
text = response.text

# Sentence split
# Download punkt_tab sentence tokenizer from the natural language toolkit
nltk.download('punkt_tab', quiet=True)
sentences = nltk.sent_tokenize(text)

# Setup GPT-4 tokenizer @ 400 tokens target
enc = tiktoken.get_encoding("cl100k_base")
token_limit = 400

# Initiate chunking index and list
chunks = []
current_chunk = ""
current_tokens = 0

# Chunk
for sentence in sentences:
    sentence_tokens = len(enc.encode(sentence))
    # If adding this sentence would go over the limit, start a new chunk
    if current_tokens + sentence_tokens > token_limit:
        if current_chunk:
            chunks.append(current_chunk)
        current_chunk = sentence
        current_tokens = sentence_tokens
    else:
        if current_chunk:
            current_chunk += " " + sentence
        else:
            current_chunk = sentence
        current_tokens += sentence_tokens

# Don't forget the last chunk!
if current_chunk:
    chunks.append(current_chunk)

print(f"Number of chunks: {len(chunks)}")
print("\nSample chunk:\n", chunks[10])
#### VDB Initialization

We'll use [ChromaDB](https://www.trychroma.com/) as our lightweight database of choice. ChromaDB by default relies on the embedding model [all-MiniLM-L6-v2](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2) for embedding documents and queries.
import chromadb

# Instantiate the Chroma Client
chroma_client = chromadb.Client()

# Create a Collection
collection = chroma_client.get_or_create_collection(name="sherlock_holmes")

# Embed Chunks to the Collection
collection.add(
    documents=chunks,
    ids=[str(i) for i in range(len(chunks))]
)
#### 1st Stage Retrieval

We'll create a simple function to handle the initial retrieval from the collection, what's performed before reranking.
def retrieve_docs(query, collection="sherlock_holmes", n=25):
    # Load Chroma Collection
    collection = chroma_client.get_or_create_collection(name=collection)

    # Perform semantic search
    results = collection.query(
        query_texts=[query],
        n_results=n
    )

    # Zip documents and distances together into dicts
    docs = results["documents"][0]
    scores = results["distances"][0]

    # Combine into list of dicts
    return [{"document": doc, "score": score} for doc, score in zip(docs, scores)]
#### 2nd Stage Reranking - Cross Encoder

The first of two reranking functions, running our retrieved through a cross encoder, using the same as in our prior example.
from sentence_transformers import CrossEncoder

cross_encoder = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

def rerank_with_cross_encoder(query, results, cross_encoder_model=cross_encoder):
    # Grab chunks from dictionary
    documents = [r['document'] for r in results]
    
    # Compute cross encoder relevancy score
    rerank_scores = cross_encoder_model.predict([(query, doc) for doc in documents])
    
    for r, score in zip(results, rerank_scores):
        r['cross_encoder_score'] = float(score)
    
    # Sort results by cross_encoder_score, descending
    results = sorted(results, key=lambda x: x['cross_encoder_score'], reverse=True)
    
    return results
#### 2nd Stage Reranking - Late Interaction

The second of our reranking functions, late interaction using ColBERT once more! We'll redefine our prior two helper functions again if this is being ran in isolation from the walkthrough code above.
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

# Load ColBERTv2 model & tokenizer just once (at top level)
colbert_tokenizer = AutoTokenizer.from_pretrained("colbert-ir/colbertv2.0")
colbert_model = AutoModel.from_pretrained("colbert-ir/colbertv2.0")

def get_token_embeddings(text, tokenizer, model):
    # Get token-level embeddings, ignore [CLS] and [SEP]
    inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
    input_ids = inputs['input_ids'][0]
    keep_indices = (input_ids != tokenizer.cls_token_id) & (input_ids != tokenizer.sep_token_id)
    return outputs.last_hidden_state[0][keep_indices]  # (filtered_seq_len, hidden_dim)

def colbert_score(query_emb, doc_emb):
    # query_emb: (m, d), doc_emb: (n, d)
    sim = F.cosine_similarity(query_emb.unsqueeze(1), doc_emb.unsqueeze(0), dim=2)  # (m, n)
    max_sim, _ = sim.max(dim=1)  # (m,)
    return max_sim.sum().item()

def rerank_with_late_interaction(query, results, tokenizer=colbert_tokenizer, model=colbert_model):
    # Precompute query embeddings once
    query_emb = get_token_embeddings(query, tokenizer, model)
    for r in results:
        doc_emb = get_token_embeddings(r['document'], tokenizer, model)
        r['late_interaction_score'] = colbert_score(query_emb, doc_emb)
    # Sort results by late_interaction_score descending
    results = sorted(results, key=lambda x: x['late_interaction_score'], reverse=True)
    return results
### Testing it Out!

Let's now run the entire pipeline and compare our outputs
**Stage 1: Retrieval only**
query = "Show moments where Holmes or Watson reflect on friendship."

# Stage 1: Retrieve top 50
retrieved_docs = retrieve_docs(query, collection="sherlock_holmes", n=50)

print("="*20, "Top 5 Retrieved (Semantic Search Only)", "="*20)
for i, r in enumerate(retrieved_docs[:5]):
    print(f"[{i+1}] Score: {r['score']:.4f}\n{r['document'][:200]}...\n")
**Stage 2: Cross-Encoder reranking**
cross_encoder_reranked = rerank_with_cross_encoder(query, retrieved_docs)

print("="*20, "Top 5 After Cross Encoder Rerank", "="*20)
for i, r in enumerate(cross_encoder_reranked[:5]):
    print(f"[{i+1}] Cross-Encoder Score: {r['cross_encoder_score']:.2f}\n{r['document'][:200]}...\n")
**Stage 3: Late Interaction reranking**
late_interaction_reranked = rerank_with_late_interaction(query, retrieved_docs)

print("="*20, "Top 5 After Late Interaction Rerank", "="*20)
for i, r in enumerate(late_interaction_reranked[:5]):
    print(f"[{i+1}] Late Interaction Score: {r['late_interaction_score']:.2f}\n{r['document'][:200]}...\n")
### Combining with an LLM for RAG

Now that we have our retrieval and reranking systems in place, we can now combine everything into a full fledged RAG system.
from openai import OpenAI

def rag_response(query, ranking="none", k=5):

    # Instantiate OpenAI Client
    client = OpenAI()

    # Retrieve our initial set of 50 documents
    retrieved_documents = retrieve_docs(query, n=50)  # returns list of dicts with 'document' (and scores)

    # Rerank and sort based on our three methods
    if ranking == "none":
        sorted_docs = sorted(retrieved_documents, key=lambda r: r['score'])
        docs = [r['document'] for r in sorted_docs[:k]]
    elif ranking == "cross_encoder":
        reranked = rerank_with_cross_encoder(query, retrieved_documents)
        docs = [r['document'] for r in reranked[:k]]
    elif ranking == "late_interaction":
        reranked = rerank_with_late_interaction(query, retrieved_documents)
        docs = [r['document'] for r in reranked[:k]]
    else:
        raise ValueError("Argument 'ranking' must be one of ['none', 'cross_encoder', 'late_interaction']")

    # DEBUG PRINT: show which chunks are being provided
    print(f"\nProvided Chunks (top {k}, ranking: {ranking}):\n" + "-"*60)
    for i, doc in enumerate(docs, 1):
        print(f"[{i}] {doc[:250]}{'...' if len(doc)>250 else ''}\n")  # Show first 250 chars for readability
    print("="*50)

    # Combine top documents for passage to LLM
    context = "\n\n".join(docs)
    prompt = f"""You are a Sherlock Holmes expert. Use ONLY the following passages from the stories to answer the question.

Passages:
{context}

Question: {query}

If you cannot find an answer in the passages, reply: "The answer is not shown in the provided context." Otherwise, answer as specifically as possible using the text above.
"""

    # Generate response from the model
    response = client.chat.completions.create(
        model="gpt-4.1",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.5
    )
    
    return response.choices[0].message.content.strip()
**Simple Semantic Search**
answer = rag_response("Show moments where Holmes or Watson reflect on friendship.", ranking="none")
print(answer)
**Reranking with Cross Encoder**
answer = rag_response("Show moments where Holmes or Watson reflect on friendship.", ranking="cross_encoder")
print(answer)
**Reranking with Late Interaction**
answer = rag_response("Show moments where Holmes or Watson reflect on friendship.", ranking="late_interaction")
print(answer)
---
## Discussion

In the above notebook we've demonstrated a two step retrieval and reranking process covering the tradition bi-encoder architecture for simple retrieval then cross-encoder and late interaction style setups and models for computing. The workings and findings of such are outlined in the below summary table:

**Comparison Table**

| Step         | Retrieval (Embedding)          | Rerank: Late Interaction (ColBERT)   | Rerank: Cross-Encoder           |
| ------------ | ------------------------------ | ------------------------------------ | ------------------------------- |
| Query Encode | 1 vector per query             | Matrix of vectors per query          | Joint encoding (query+doc pair) |
| Doc Encode   | 1 vector per doc (pre-compute) | Matrix of vectors per doc (pre-comp) | N/A (encode per query+doc pair) |
| Scoring      | Cosine/dot similarity          | MaxSim per query token + aggregate   | Full transformer, \[CLS] output |
| Compute cost | Very low                       | Moderate (matrix op per pair)        | High (full forward per pair)    |

In essence, the bi-encoder is a recall-oriented coarse retrieval tool, while the cross-encoder and late interaction re-ranker is a precision-oriented fine reranking step. By relying on the traditional bi-encoder to quickly create a subset that can then be refined by a more compute intensive reranking model and improve retrieval results and downstream LLM generated contextual responses.

In [ ]:
!pip install sentence-transformers transformers chromadb nltk torch beautifulsoup4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.7 MB/s et

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
The token `read` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `read`


In [ ]:
# Load local models instead of cloud-based ones
from sentence_transformers import SentenceTransformer, CrossEncoder
from transformers import AutoTokenizer, AutoModel
import chromadb
import torch
import torch.nn.functional as F
import nltk
from transformers import pipeline  # For local LLM
import os

# Download resources once
nltk.download('punkt', quiet=True)

# ======== LOCAL MODELS CONFIGURATION ========
# Bi-encoder for embeddings
embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Cross-encoder for reranking
cross_encoder = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

# ColBERT for late interaction
colbert_tokenizer = AutoTokenizer.from_pretrained("colbert-ir/colbertv2.0")
colbert_model = AutoModel.from_pretrained("colbert-ir/colbertv2.0")

# Local LLM for generation (using Zephyr 7B Beta - smaller alternative: 'gpt2')
local_llm = pipeline("text-generation",
                     model="meta-llama/Llama-3.2-1B-Instruct",
                     device_map="auto",
                     torch_dtype=torch.bfloat16)

# ======== TEXT CHUNKING ========
def prepare_documents():
    """Prepare documents for vector database"""
    import requests
    from bs4 import BeautifulSoup

    # Download Sherlock Holmes text
    print("Downloading Sherlock Holmes text...")
    url = "https://www.gutenberg.org/files/1661/1661-0.txt"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    text = soup.get_text()

    # Sentence splitting
    sentences = nltk.sent_tokenize(text)

    # Simple chunking
    chunks = []
    current_chunk = ""
    token_limit = 400  # Target token count

    for sentence in sentences:
        # Approximate token count (1 token ~ 4 chars)
        sentence_tokens = len(sentence) // 4
        if len(current_chunk) + sentence_tokens > token_limit:
            if current_chunk:
                chunks.append(current_chunk)
            current_chunk = sentence
        else:
            current_chunk += " " + sentence if current_chunk else sentence

    if current_chunk:
        chunks.append(current_chunk)

    print(f"Created {len(chunks)} chunks")
    return chunks

# Initialize ChromaDB
chroma_client = chromadb.PersistentClient(path="./chroma_db")
collection = chroma_client.get_or_create_collection(
    name="sherlock_holmes",
    metadata={"hnsw:space": "cosine"}
)

# Add documents to ChromaDB if not already loaded
if collection.count() == 0:
    print("Populating vector database...")
    chunks = prepare_documents()
    embeddings = embedding_model.encode(chunks).tolist()
    collection.add(
        documents=chunks,
        embeddings=embeddings,
        ids=[str(i) for i in range(len(chunks))]
    )

# ======== HELPER FUNCTIONS ========
def get_token_embeddings(text, tokenizer, model):
    """Get token-level embeddings"""
    inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    input_ids = inputs['input_ids'][0]
    keep_indices = (input_ids != tokenizer.cls_token_id) & (input_ids != tokenizer.sep_token_id)
    return outputs.last_hidden_state[0][keep_indices]

def colbert_score(query_emb, doc_emb):
    """Compute ColBERT late interaction score"""
    sim = F.cosine_similarity(query_emb.unsqueeze(1), doc_emb.unsqueeze(0), dim=2)
    max_sim, _ = sim.max(dim=1)
    return max_sim.sum().item()

# ======== RETRIEVAL FUNCTIONS ========
def retrieve_docs(query, n=25):
    """Initial retrieval from vector database"""
    query_embedding = embedding_model.encode(query).tolist()
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=n
    )
    return [
        {"document": doc, "score": score}
        for doc, score in zip(results['documents'][0], results['distances'][0])
    ]

def rerank_with_cross_encoder(query, results):
    """Rerank using cross-encoder"""
    documents = [r['document'] for r in results]
    rerank_scores = cross_encoder.predict([(query, doc) for doc in documents])
    for r, score in zip(results, rerank_scores):
        r['cross_encoder_score'] = float(score)
    return sorted(results, key=lambda x: x['cross_encoder_score'], reverse=True)

def rerank_with_late_interaction(query, results):
    """Rerank using late interaction"""
    query_emb = get_token_embeddings(query, colbert_tokenizer, colbert_model)
    for r in results:
        doc_emb = get_token_embeddings(r['document'], colbert_tokenizer, colbert_model)
        r['late_interaction_score'] = colbert_score(query_emb, doc_emb)
    return sorted(results, key=lambda x: x['late_interaction_score'], reverse=True)

# ======== LOCAL LLM RESPONSE GENERATION ========
def generate_local_response(prompt):
    """Generate response using local LLM"""
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]

    response = local_llm(
        messages,
        max_new_tokens=500,
        temperature=0.7,
        do_sample=True,
        pad_token_id=local_llm.tokenizer.eos_token_id
    )

    return response[0]['generated_text'][-1]['content']

def rag_response(query, ranking="none", k=5):
    """Full RAG pipeline with local models"""
    # Retrieve initial documents
    retrieved_documents = retrieve_docs(query, n=50)

    # Apply selected reranking
    if ranking == "cross_encoder":
        reranked = rerank_with_cross_encoder(query, retrieved_documents)
        docs = [r['document'] for r in reranked[:k]]
    elif ranking == "late_interaction":
        reranked = rerank_with_late_interaction(query, retrieved_documents)
        docs = [r['document'] for r in reranked[:k]]
    else:  # No reranking
        docs = [r['document'] for r in retrieved_documents[:k]]

    # Prepare context
    context = "\n\n".join(docs)

    # Create prompt for LLM
    prompt = f"""Use the following passages to answer the question. If the answer isn't in the context, say so.

Context:
{context}

Question: {query}

Answer:"""

    # Generate response
    return generate_local_response(prompt)

# ======== EXAMPLE USAGE ========
if __name__ == "__main__":
    # Example query
    query = "Show moments where Holmes or Watson reflect on friendship."

    print("\n=== RAG Response with Semantic Search Only ===")
    print(rag_response(query, ranking="none"))

    print("\n=== RAG Response with Cross-Encoder Reranking ===")
    print(rag_response(query, ranking="cross_encoder"))

    print("\n=== RAG Response with Late Interaction Reranking ===")
    print(rag_response(query, ranking="late_interaction"))

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Device set to use cpu


Populating vector database...


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [ ]:
# نظام RAG محلي بالكامل باستخدام نماذج مجانية
from sentence_transformers import SentenceTransformer, CrossEncoder
from transformers import AutoTokenizer, AutoModel, pipeline
import chromadb
import torch
import torch.nn.functional as F
import nltk
import requests
from bs4 import BeautifulSoup
import re
import os

# ======== إعداد البيئة ========
# تنزيل حزمة nltk المطلوبة
try:
    nltk.data.find('tokenizers/punkt')
    print("nltk punkt package already installed")
except LookupError:
    print("Downloading nltk punkt package...")
    nltk.download('punkt', quiet=True)

# ======== تهيئة النماذج ========
print("Loading models...")

# نموذج التضمين للبحث الدلالي
embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# نموذج إعادة التصنيف Cross-Encoder
cross_encoder = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

# نموذج إعادة التصنيف ColBERT
colbert_tokenizer = AutoTokenizer.from_pretrained("colbert-ir/colbertv2.0")
colbert_model = AutoModel.from_pretrained("colbert-ir/colbertv2.0")

# نموذج توليد النصوص المحلي (بديل أصغر: 'gpt2')
local_llm = pipeline(
    "text-generation",
    model="meta-llama/Llama-3.2-1B-Instruct",
    device_map="auto",
    torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32
)

print("All models loaded successfully!")

# ======== إعداد قاعدة البيانات المتجهة ========
def prepare_documents():
    """تحضير المستندات لقاعدة البيانات المتجهة"""
    print("Downloading Sherlock Holmes text...")
    url = "https://www.gutenberg.org/files/1661/1661-0.txt"
    response = requests.get(url)
    text = response.text

    # تنظيف النص من الرأس والتذييل
    start_idx = text.find("ADVENTURE I. A SCANDAL IN BOHEMIA")
    end_idx = text.find("End of the Project Gutenberg")
    text = text[start_idx:end_idx] if start_idx != -1 and end_idx != -1 else text

    # تقسيم النص إلى جمل
    sentences = nltk.sent_tokenize(text)

    # تجميع الجمل في مقاطع
    chunks = []
    current_chunk = ""
    token_limit = 400  # حد الرموز المستهدف
    char_per_token = 4  # تقدير: 4 أحرف لكل رمز

    for sentence in sentences:
        # حساب الرموز التقريبي
        sentence_tokens = len(sentence) / char_per_token

        if len(current_chunk) + sentence_tokens > token_limit:
            if current_chunk:
                chunks.append(current_chunk)
            current_chunk = sentence
        else:
            current_chunk += " " + sentence if current_chunk else sentence

    if current_chunk:
        chunks.append(current_chunk)

    print(f"Created {len(chunks)} chunks")
    return chunks

# تهيئة قاعدة البيانات المتجهة
chroma_client = chromadb.PersistentClient(path="./chroma_db")
collection = chroma_client.get_or_create_collection(
    name="sherlock_holmes",
    metadata={"hnsw:space": "cosine"}
)

# إضافة المستندات إذا لم تكن موجودة
if collection.count() == 0:
    print("Populating vector database...")
    chunks = prepare_documents()
    embeddings = embedding_model.encode(chunks).tolist()
    ids = [str(i) for i in range(len(chunks))]

    collection.add(
        documents=chunks,
        embeddings=embeddings,
        ids=ids
    )
    print(f"Added {len(chunks)} documents to vector database")
else:
    print(f"Vector database already contains {collection.count()} documents")

# ======== دوال المساعدة ========
def get_token_embeddings(text, tokenizer, model):
    """الحصول على تمثيلات الرموز"""
    inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    input_ids = inputs['input_ids'][0]
    # تجاهل الرموز الخاصة [CLS] و [SEP]
    keep_indices = (input_ids != tokenizer.cls_token_id) & (input_ids != tokenizer.sep_token_id)
    return outputs.last_hidden_state[0][keep_indices]

def colbert_score(query_emb, doc_emb):
    """حساب درجة التشابه باستخدام ColBERT"""
    # حساب التشابه بين كل رمز في الاستعلام وكل رمز في المستند
    sim = F.cosine_similarity(query_emb.unsqueeze(1), doc_emb.unsqueeze(0), dim=2)
    # أخذ أقصى تشابه لكل رمز في الاستعلام
    max_sim, _ = sim.max(dim=1)
    # جمع القيم القصوى
    return max_sim.sum().item()

# ======== دوال الاسترجاع وإعادة التصنيف ========
def retrieve_docs(query, n=25):
    """الاسترجاع الأولي من قاعدة البيانات"""
    query_embedding = embedding_model.encode(query).tolist()
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=n
    )
    return [
        {"document": doc, "score": score}
        for doc, score in zip(results['documents'][0], results['distances'][0])
    ]

def rerank_with_cross_encoder(query, results):
    """إعادة تصنيف النتائج باستخدام Cross-Encoder"""
    documents = [r['document'] for r in results]
    # حساب درجات الأهمية لكل مستند بالنسبة للاستعلام
    rerank_scores = cross_encoder.predict([(query, doc) for doc in documents])
    for r, score in zip(results, rerank_scores):
        r['cross_encoder_score'] = float(score)
    # ترتيب النتائج تنازلياً حسب الدرجة
    return sorted(results, key=lambda x: x['cross_encoder_score'], reverse=True)

def rerank_with_late_interaction(query, results):
    """إعادة تصنيف النتائج باستخدام التفاعل المتأخر (ColBERT)"""
    # تمثيل الرموز للاستعلام
    query_emb = get_token_embeddings(query, colbert_tokenizer, colbert_model)
    for r in results:
        # تمثيل الرموز لكل مستند
        doc_emb = get_token_embeddings(r['document'], colbert_tokenizer, colbert_model)
        # حساب درجة التفاعل المتأخر
        r['late_interaction_score'] = colbert_score(query_emb, doc_emb)
    # ترتيب النتائج تنازلياً حسب الدرجة
    return sorted(results, key=lambda x: x['late_interaction_score'], reverse=True)

# ======== توليد الإجابة ========
def generate_local_response(prompt, max_length=500):
    """توليد الإجابة باستخدام نموذج محلي"""
    # بناء رسائل المحادثة
    messages = [
        {"role": "system", "content": "You are a helpful AI assistant."},
        {"role": "user", "content": prompt}
    ]

    # توليد الاستجابة
    response = local_llm(
        messages,
        max_new_tokens=max_length,
        temperature=0.7,
        do_sample=True,
        pad_token_id=local_llm.tokenizer.eos_token_id
    )

    # استخراج المحتوى النهائي للإجابة
    return response[0]['generated_text'][-1]['content']

def rag_response(query, ranking="none", k=5):
    """نظام RAG الكامل"""
    print(f"\nProcessing query: '{query}'")

    # 1. الاسترجاع الأولي
    print("Retrieving initial documents...")
    retrieved_documents = retrieve_docs(query, n=50)

    # 2. إعادة التصنيف (اختياري)
    if ranking == "cross_encoder":
        print("Reranking with cross-encoder...")
        reranked = rerank_with_cross_encoder(query, retrieved_documents)
        docs = [r['document'] for r in reranked[:k]]
    elif ranking == "late_interaction":
        print("Reranking with late interaction...")
        reranked = rerank_with_late_interaction(query, retrieved_documents)
        docs = [r['document'] for r in reranked[:k]]
    else:  # بدون إعادة تصنيف
        docs = [r['document'] for r in retrieved_documents[:k]]

    # 3. تحضير السياق
    context = "\n\n".join([f"Document {i+1}: {doc}" for i, doc in enumerate(docs)])

    # 4. بناء سؤال الذكاء الاصطناعي
    prompt = f"""استخدم المقاطع التالية للإجابة على السؤال. إذا لم تجد الإجابة في المقاطع، قل "لا أعرف".

المقاطع:
{context}

السؤال: {query}

الإجابة:"""

    # 5. توليد الإجابة
    print("Generating response...")
    return generate_local_response(prompt)

# ======== الواجهة الرئيسية ========
def main():
    print("\n" + "="*50)
    print("نظام RAG المحلي بالكامل - مغامرات شرلوك هولمز")
    print("="*50 + "\n")

    while True:
        query = input("\nأدخل سؤالك (أو اكتب 'خروج' للإنهاء): ")

        if query.lower() in ['exit', 'quit', 'خروج']:
            break

        print("\n1. البحث الدلالي فقط")
        print("2. البحث مع إعادة التصنيف (Cross-Encoder)")
        print("3. البحث مع إعادة التصنيف (Late Interaction)")
        choice = input("اختر طريقة البحث (1/2/3): ")

        if choice == '1':
            response = rag_response(query, ranking="none")
        elif choice == '2':
            response = rag_response(query, ranking="cross_encoder")
        elif choice == '3':
            response = rag_response(query, ranking="late_interaction")
        else:
            print("اختيار غير صحيح، استخدام البحث الدلالي فقط")
            response = rag_response(query, ranking="none")

        print("\n" + "="*50)
        print("الإجابة:")
        print(response)
        print("="*50)

if __name__ == "__main__":
    main()

nltk punkt package already installed
Loading models...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cpu


All models loaded successfully!
Populating vector database...


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


### شغال

In [ ]:
# نظام RAG محلي بالكامل باستخدام نماذج مجانية - الإصدار المعدل
from sentence_transformers import SentenceTransformer, CrossEncoder
from transformers import AutoTokenizer, AutoModel, pipeline
import chromadb
import torch
import torch.nn.functional as F
import requests
import re
import os

# ======== إعداد البيئة ========
print("Loading models...")

# نموذج التضمين للبحث الدلالي
embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# نموذج إعادة التصنيف Cross-Encoder
cross_encoder = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

# نموذج إعادة التصنيف ColBERT
colbert_tokenizer = AutoTokenizer.from_pretrained("colbert-ir/colbertv2.0")
colbert_model = AutoModel.from_pretrained("colbert-ir/colbertv2.0")

# نموذج توليد النصوص المحلي (بديل أصغر: 'gpt2')
local_llm = pipeline(
    "text-generation",
    model="meta-llama/Llama-3.2-1B-Instruct",
    device_map="auto",
    torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32
)

print("All models loaded successfully!")

# ======== إعداد قاعدة البيانات المتجهة ========
def split_sentences(text):
    """تقسيم النص إلى جمل باستخدام regex بدون الحاجة لـ nltk"""
    # تقسيم عند علامات الترقيم مع الحفاظ على الاختصارات
    pattern = r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|\!)\s'
    return re.split(pattern, text)

def prepare_documents():
    """تحضير المستندات لقاعدة البيانات المتجهة"""
    print("Downloading Sherlock Holmes text...")
    url = "https://www.gutenberg.org/files/1661/1661-0.txt"
    response = requests.get(url)
    text = response.text

    # تنظيف النص من الرأس والتذييل
    start_idx = text.find("ADVENTURE I. A SCANDAL IN BOHEMIA")
    end_idx = text.find("End of the Project Gutenberg")
    text = text[start_idx:end_idx] if start_idx != -1 and end_idx != -1 else text

    # تقسيم النص إلى جمل
    sentences = split_sentences(text)

    # تجميع الجمل في مقاطع
    chunks = []
    current_chunk = ""
    token_limit = 400  # حد الرموز المستهدف
    char_per_token = 4  # تقدير: 4 أحرف لكل رمز

    for sentence in sentences:
        # حساب الرموز التقريبي
        sentence_tokens = len(sentence) / char_per_token

        if len(current_chunk) + sentence_tokens > token_limit:
            if current_chunk:
                chunks.append(current_chunk)
            current_chunk = sentence
        else:
            current_chunk += " " + sentence if current_chunk else sentence

    if current_chunk:
        chunks.append(current_chunk)

    print(f"Created {len(chunks)} chunks")
    return chunks

# تهيئة قاعدة البيانات المتجهة
chroma_client = chromadb.PersistentClient(path="./chroma_db")
collection = chroma_client.get_or_create_collection(
    name="sherlock_holmes",
    metadata={"hnsw:space": "cosine"}
)

# إضافة المستندات إذا لم تكن موجودة
if collection.count() == 0:
    print("Populating vector database...")
    chunks = prepare_documents()
    embeddings = embedding_model.encode(chunks).tolist()
    ids = [str(i) for i in range(len(chunks))]

    collection.add(
        documents=chunks,
        embeddings=embeddings,
        ids=ids
    )
    print(f"Added {len(chunks)} documents to vector database")
else:
    print(f"Vector database already contains {collection.count()} documents")

# ======== دوال المساعدة ========
def get_token_embeddings(text, tokenizer, model):
    """الحصول على تمثيلات الرموز"""
    inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    input_ids = inputs['input_ids'][0]
    # تجاهل الرموز الخاصة [CLS] و [SEP]
    keep_indices = (input_ids != tokenizer.cls_token_id) & (input_ids != tokenizer.sep_token_id)
    return outputs.last_hidden_state[0][keep_indices]

def colbert_score(query_emb, doc_emb):
    """حساب درجة التشابه باستخدام ColBERT"""
    # حساب التشابه بين كل رمز في الاستعلام وكل رمز في المستند
    sim = F.cosine_similarity(query_emb.unsqueeze(1), doc_emb.unsqueeze(0), dim=2)
    # أخذ أقصى تشابه لكل رمز في الاستعلام
    max_sim, _ = sim.max(dim=1)
    # جمع القيم القصوى
    return max_sim.sum().item()

# ======== دوال الاسترجاع وإعادة التصنيف ========
def retrieve_docs(query, n=25):
    """الاسترجاع الأولي من قاعدة البيانات"""
    query_embedding = embedding_model.encode(query).tolist()
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=n
    )
    return [
        {"document": doc, "score": score}
        for doc, score in zip(results['documents'][0], results['distances'][0])
    ]

def rerank_with_cross_encoder(query, results):
    """إعادة تصنيف النتائج باستخدام Cross-Encoder"""
    documents = [r['document'] for r in results]
    # حساب درجات الأهمية لكل مستند بالنسبة للاستعلام
    rerank_scores = cross_encoder.predict([(query, doc) for doc in documents])
    for r, score in zip(results, rerank_scores):
        r['cross_encoder_score'] = float(score)
    # ترتيب النتائج تنازلياً حسب الدرجة
    return sorted(results, key=lambda x: x['cross_encoder_score'], reverse=True)

def rerank_with_late_interaction(query, results):
    """إعادة تصنيف النتائج باستخدام التفاعل المتأخر (ColBERT)"""
    # تمثيل الرموز للاستعلام
    query_emb = get_token_embeddings(query, colbert_tokenizer, colbert_model)
    for r in results:
        # تمثيل الرموز لكل مستند
        doc_emb = get_token_embeddings(r['document'], colbert_tokenizer, colbert_model)
        # حساب درجة التفاعل المتأخر
        r['late_interaction_score'] = colbert_score(query_emb, doc_emb)
    # ترتيب النتائج تنازلياً حسب الدرجة
    return sorted(results, key=lambda x: x['late_interaction_score'], reverse=True)

# ======== توليد الإجابة ========
def generate_local_response(prompt, max_length=500):
    """توليد الإجابة باستخدام نموذج محلي"""
    # بناء رسائل المحادثة
    messages = [
        {"role": "system", "content": "You are a helpful AI assistant."},
        {"role": "user", "content": prompt}
    ]

    # توليد الاستجابة
    response = local_llm(
        messages,
        max_new_tokens=max_length,
        temperature=0.7,
        do_sample=True,
        pad_token_id=local_llm.tokenizer.eos_token_id
    )

    # استخراج المحتوى النهائي للإجابة
    generated_text = response[0]['generated_text']

    # في بعض الأحيان يرجع النموذج الرسائل كاملة، لذلك نستخرج فقط الرد الأخير
    if "assistant" in generated_text:
        # استخراج الجزء بعد آخر "assistant"
        parts = generated_text.split("assistant")
        if len(parts) > 1:
            return parts[-1].strip()

    # إذا لم نتمكن من استخراج الرد، نرجع النص كاملاً
    return generated_text

def rag_response(query, ranking="none", k=5):
    """نظام RAG الكامل"""
    print(f"\nProcessing query: '{query}'")

    # 1. الاسترجاع الأولي
    print("Retrieving initial documents...")
    retrieved_documents = retrieve_docs(query, n=50)

    # 2. إعادة التصنيف (اختياري)
    if ranking == "cross_encoder":
        print("Reranking with cross-encoder...")
        reranked = rerank_with_cross_encoder(query, retrieved_documents)
        docs = [r['document'] for r in reranked[:k]]
    elif ranking == "late_interaction":
        print("Reranking with late interaction...")
        reranked = rerank_with_late_interaction(query, retrieved_documents)
        docs = [r['document'] for r in reranked[:k]]
    else:  # بدون إعادة تصنيف
        docs = [r['document'] for r in retrieved_documents[:k]]

    # 3. تحضير السياق
    context = "\n\n".join([f"Document {i+1}: {doc}" for i, doc in enumerate(docs)])

    # 4. بناء سؤال الذكاء الاصطناعي
    prompt = f"""Use the following passages to answer the question. If you don't find the answer in the passages, say "I don't know."

المقاطع:
{context}

السؤال: {query}

الإجابة:"""

    # 5. توليد الإجابة
    print("Generating response...")
    return generate_local_response(prompt)

# ======== الواجهة الرئيسية ========
def main():
    print("\n" + "="*50)
    print("نظام RAG المحلي بالكامل - مغامرات شرلوك هولمز")
    print("="*50 + "\n")

    while True:
        query = input("\nأدخل سؤالك (أو اكتب 'خروج' للإنهاء): ")

        if query.lower() in ['exit', 'quit', 'خروج']:
            print("شكراً لاستخدامك النظام. إلى اللقاء!")
            break

        print("\n1. البحث الدلالي فقط")
        print("2. البحث مع إعادة التصنيف (Cross-Encoder)")
        print("3. البحث مع إعادة التصنيف (Late Interaction)")
        choice = input("اختر طريقة البحث (1/2/3): ")

        if choice == '1':
            response = rag_response(query, ranking="none")
        elif choice == '2':
            response = rag_response(query, ranking="cross_encoder")
        elif choice == '3':
            response = rag_response(query, ranking="late_interaction")
        else:
            print("اختيار غير صحيح، استخدام البحث الدلالي فقط")
            response = rag_response(query, ranking="none")

        print("\n" + "="*50)
        print("الإجابة:")
        print(response)
        print("="*50)

if __name__ == "__main__":
    main()

Loading models...


Device set to use cpu


All models loaded successfully!
Populating vector database...
Created 1328 chunks
Added 1328 documents to vector database

نظام RAG المحلي بالكامل - مغامرات شرلوك هولمز


1. البحث الدلالي فقط
2. البحث مع إعادة التصنيف (Cross-Encoder)
3. البحث مع إعادة التصنيف (Late Interaction)

Processing query: 'Who is Sherlock Holmes?'
Retrieving initial documents...
Generating response...

الإجابة:
[{'role': 'system', 'content': 'You are a helpful AI assistant.'}, {'role': 'user', 'content': 'استخدم المقاطع التالية للإجابة على السؤال. إذا لم تجد الإجابة في المقاطع، قل "لا أعرف".\n\nالمقاطع:\nDocument 1: \n\r\nSherlock Holmes sat silent for some few minutes, with his brows knitted\r\nand his eyes fixed upon the fire. \n\r\n“Do you receive much company?” he asked. \n\r\n“None save my partner with his family and an occasional friend of\r\nArthur’s. Sir George Burnwell has been several times lately. No one\r\nelse, I think.”\r\n\r\n“Do you go out much in society?”\r\n\r\n“Arthur does. Mary and I stay a

In [1]:
# نظام RAG محلي بالكامل باستخدام نماذج مجانية - الإصدار المعدل
from sentence_transformers import SentenceTransformer, CrossEncoder
from transformers import AutoTokenizer, AutoModel, pipeline
import chromadb
import torch
import torch.nn.functional as F
import requests
import re
import os

# ======== إعداد البيئة ========
print("Loading models...")

# نموذج التضمين للبحث الدلالي
embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# نموذج إعادة التصنيف Cross-Encoder
cross_encoder = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

# نموذج إعادة التصنيف ColBERT
colbert_tokenizer = AutoTokenizer.from_pretrained("colbert-ir/colbertv2.0")
colbert_model = AutoModel.from_pretrained("colbert-ir/colbertv2.0")

# نموذج توليد النصوص المحلي (بديل أصغر: 'gpt2')
local_llm = pipeline(
    "text-generation",
    model="meta-llama/Llama-3.2-1B-Instruct",
    device_map="auto",
    torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32
)

print("All models loaded successfully!")

# ======== إعداد قاعدة البيانات المتجهة ========
def split_sentences(text):
    """تقسيم النص إلى جمل باستخدام regex بدون الحاجة لـ nltk"""
    # تقسيم عند علامات الترقيم مع الحفاظ على الاختصارات
    pattern = r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|\!)\s'
    return re.split(pattern, text)

def prepare_documents():
    """تحضير المستندات لقاعدة البيانات المتجهة"""
    print("Downloading Sherlock Holmes text...")
    url = "https://www.gutenberg.org/files/1661/1661-0.txt"
    response = requests.get(url)
    text = response.text

    # تنظيف النص من الرأس والتذييل
    start_idx = text.find("ADVENTURE I. A SCANDAL IN BOHEMIA")
    end_idx = text.find("End of the Project Gutenberg")
    text = text[start_idx:end_idx] if start_idx != -1 and end_idx != -1 else text

    # تقسيم النص إلى جمل
    sentences = split_sentences(text)

    # تجميع الجمل في مقاطع
    chunks = []
    current_chunk = ""
    token_limit = 400  # حد الرموز المستهدف
    char_per_token = 4  # تقدير: 4 أحرف لكل رمز

    for sentence in sentences:
        # حساب الرموز التقريبي
        sentence_tokens = len(sentence) / char_per_token

        if len(current_chunk) + sentence_tokens > token_limit:
            if current_chunk:
                chunks.append(current_chunk)
            current_chunk = sentence
        else:
            current_chunk += " " + sentence if current_chunk else sentence

    if current_chunk:
        chunks.append(current_chunk)

    print(f"Created {len(chunks)} chunks")
    return chunks

# تهيئة قاعدة البيانات المتجهة
chroma_client = chromadb.PersistentClient(path="./chroma_db")
collection = chroma_client.get_or_create_collection(
    name="sherlock_holmes",
    metadata={"hnsw:space": "cosine"}
)

# إضافة المستندات إذا لم تكن موجودة
if collection.count() == 0:
    print("Populating vector database...")
    chunks = prepare_documents()
    embeddings = embedding_model.encode(chunks).tolist()
    ids = [str(i) for i in range(len(chunks))]

    collection.add(
        documents=chunks,
        embeddings=embeddings,
        ids=ids
    )
    print(f"Added {len(chunks)} documents to vector database")
else:
    print(f"Vector database already contains {collection.count()} documents")

# ======== دوال المساعدة ========
def get_token_embeddings(text, tokenizer, model):
    """الحصول على تمثيلات الرموز"""
    inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    input_ids = inputs['input_ids'][0]
    # تجاهل الرموز الخاصة [CLS] و [SEP]
    keep_indices = (input_ids != tokenizer.cls_token_id) & (input_ids != tokenizer.sep_token_id)
    return outputs.last_hidden_state[0][keep_indices]

def colbert_score(query_emb, doc_emb):
    """حساب درجة التشابه باستخدام ColBERT"""
    # حساب التشابه بين كل رمز في الاستعلام وكل رمز في المستند
    sim = F.cosine_similarity(query_emb.unsqueeze(1), doc_emb.unsqueeze(0), dim=2)
    # أخذ أقصى تشابه لكل رمز في الاستعلام
    max_sim, _ = sim.max(dim=1)
    # جمع القيم القصوى
    return max_sim.sum().item()

# ======== دوال الاسترجاع وإعادة التصنيف ========
def retrieve_docs(query, n=25):
    """الاسترجاع الأولي من قاعدة البيانات"""
    query_embedding = embedding_model.encode(query).tolist()
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=n
    )
    return [
        {"document": doc, "score": score}
        for doc, score in zip(results['documents'][0], results['distances'][0])
    ]

def rerank_with_cross_encoder(query, results):
    """إعادة تصنيف النتائج باستخدام Cross-Encoder"""
    documents = [r['document'] for r in results]
    # حساب درجات الأهمية لكل مستند بالنسبة للاستعلام
    rerank_scores = cross_encoder.predict([(query, doc) for doc in documents])
    for r, score in zip(results, rerank_scores):
        r['cross_encoder_score'] = float(score)
    # ترتيب النتائج تنازلياً حسب الدرجة
    return sorted(results, key=lambda x: x['cross_encoder_score'], reverse=True)

def rerank_with_late_interaction(query, results):
    """إعادة تصنيف النتائج باستخدام التفاعل المتأخر (ColBERT)"""
    # تمثيل الرموز للاستعلام
    query_emb = get_token_embeddings(query, colbert_tokenizer, colbert_model)
    for r in results:
        # تمثيل الرموز لكل مستند
        doc_emb = get_token_embeddings(r['document'], colbert_tokenizer, colbert_model)
        # حساب درجة التفاعل المتأخر
        r['late_interaction_score'] = colbert_score(query_emb, doc_emb)
    # ترتيب النتائج تنازلياً حسب الدرجة
    return sorted(results, key=lambda x: x['late_interaction_score'], reverse=True)

# ======== توليد الإجابة ========
def generate_local_response(prompt, max_length=500):
    """توليد الإجابة باستخدام نموذج محلي"""
    # بناء رسائل المحادثة
    messages = [
        {"role": "system", "content": "You are a helpful AI assistant."},
        {"role": "user", "content": prompt}
    ]

    # توليد الاستجابة
    response = local_llm(
        messages,
        max_new_tokens=max_length,
        temperature=0.7,
        do_sample=True,
        pad_token_id=local_llm.tokenizer.eos_token_id
    )

    # استخراج المحتوى النهائي للإجابة
    generated_text = response[0]['generated_text']

    # في بعض الأحيان يرجع النموذج الرسائل كاملة، لذلك نستخرج فقط الرد الأخير
    if "assistant" in generated_text:
        # استخراج الجزء بعد آخر "assistant"
        parts = generated_text.split("assistant")
        if len(parts) > 1:
            return parts[-1].strip()

    # إذا لم نتمكن من استخراج الرد، نرجع النص كاملاً
    return generated_text

def rag_response(query, ranking="none", k=5):
    """نظام RAG الكامل"""
    print(f"\nProcessing query: '{query}'")

    # 1. الاسترجاع الأولي
    print("Retrieving initial documents...")
    retrieved_documents = retrieve_docs(query, n=50)

    # 2. إعادة التصنيف (اختياري)
    if ranking == "cross_encoder":
        print("Reranking with cross-encoder...")
        reranked = rerank_with_cross_encoder(query, retrieved_documents)
        docs = [r['document'] for r in reranked[:k]]
    elif ranking == "late_interaction":
        print("Reranking with late interaction...")
        reranked = rerank_with_late_interaction(query, retrieved_documents)
        docs = [r['document'] for r in reranked[:k]]
    else:  # بدون إعادة تصنيف
        docs = [r['document'] for r in retrieved_documents[:k]]

    # 3. تحضير السياق
    context = "\n\n".join([f"Document {i+1}: {doc}" for i, doc in enumerate(docs)])

    # 4. بناء سؤال الذكاء الاصطناعي
    prompt = f"""Use the following passages to answer the question. If you don't find the answer in the passages, say "I don't know."

المقاطع:
{context}

السؤال: {query}

الإجابة:"""

    # 5. توليد الإجابة
    print("Generating response...")
    return generate_local_response(prompt)

# ======== الواجهة الرئيسية ========
def main():
    print("\n" + "="*50)
    print("نظام RAG المحلي بالكامل - مغامرات شرلوك هولمز")
    print("="*50 + "\n")

    while True:
        query = input("\nأدخل سؤالك (أو اكتب 'خروج' للإنهاء): ")

        if query.lower() in ['exit', 'quit', 'خروج']:
            print("شكراً لاستخدامك النظام. إلى اللقاء!")
            break

        print("\n1. البحث الدلالي فقط")
        print("2. البحث مع إعادة التصنيف (Cross-Encoder)")
        print("3. البحث مع إعادة التصنيف (Late Interaction)")
        choice = input("اختر طريقة البحث (1/2/3): ")

        if choice == '1':
            response = rag_response(query, ranking="none")
        elif choice == '2':
            response = rag_response(query, ranking="cross_encoder")
        elif choice == '3':
            response = rag_response(query, ranking="late_interaction")
        else:
            print("اختيار غير صحيح، استخدام البحث الدلالي فقط")
            response = rag_response(query, ranking="none")

        print("\n" + "="*50)
        print("الإجابة:")
        print(response)
        print("="*50)

if __name__ == "__main__":
    main()

Loading models...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cpu


All models loaded successfully!
Vector database already contains 1328 documents

نظام RAG المحلي بالكامل - مغامرات شرلوك هولمز


أدخل سؤالك (أو اكتب 'خروج' للإنهاء): What is the name of Sherlock's friend?

1. البحث الدلالي فقط
2. البحث مع إعادة التصنيف (Cross-Encoder)
3. البحث مع إعادة التصنيف (Late Interaction)
اختر طريقة البحث (1/2/3): 1

Processing query: 'What is the name of Sherlock's friend?'
Retrieving initial documents...
Generating response...

الإجابة:
[{'role': 'system', 'content': 'You are a helpful AI assistant.'}, {'role': 'user', 'content': 'Use the following passages to answer the question. If you don\'t find the answer in the passages, say "I don\'t know."\n\nالمقاطع:\nDocument 1: \n\r\nSherlock Holmes sat silent for some few minutes, with his brows knitted\r\nand his eyes fixed upon the fire. \n\r\n“Do you receive much company?” he asked. \n\r\n“None save my partner with his family and an occasional friend of\r\nArthur’s. Sir George Burnwell has been several times lat

In [ ]:
do you think that Neville is alive?
What is the name of Sherlock's friend?
What is Sherlock's family name?
Tell me the names of Sherlock's adventures